# Download encrypted datasets

In this example, we will download 2 encrypted datasets, one coming from Azure and one from AWS.
The datasets are encrypted and also the connection with the cloud is secured, so here we show **data in transit** security.
This means that if an attacker tries to sniff the network and captures this data while it is travelling, no data will be actually leaked since data is encrypted and only the sender and recipient can decrypt it.

In [ ]:
! mkdir -p downloaded_datasets

## Azure

In [ ]:
pip install azure.storage.blob > /dev/null

In [ ]:
from azure.storage.blob import BlobServiceClient

connection_string = ""
container_name = "data1"
blob_name = "dataset1.csv.enc"
download_file_path = "downloaded_datasets/dataset1.csv.enc"

# Create the BlobServiceClient
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

# Get the container client
container_client = blob_service_client.get_container_client(container_name)

# # Download the blob
with open(download_file_path, "wb") as f:
    blob_data = container_client.download_blob(blob_name)
    f.write(blob_data.readall())
    print(f"Downloaded {blob_name}")

! head $download_file_path

## AWS

In [ ]:
pip install boto3 botocore > /dev/null

In [ ]:
import boto3
import os

# Replace with your actual AWS credentials
aws_access_key_id = 'YOUR_ACCESS_KEY_ID'
aws_secret_access_key = 'YOUR_SECRET_ACCESS_KEY'

# Replace with your bucket name
bucket_name = 'eesposit'
prefix = 'dataset2.csv.enc'
download_path = 'downloaded_datasets'

s3 = boto3.client(
    's3',
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key
)

response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)

if 'Contents' in response:
    for obj in response['Contents']:
        key = obj['Key']
        local_file_path = os.path.join(download_path, key)
        os.makedirs(os.path.dirname(local_file_path), exist_ok=True)
        s3.download_file(bucket_name, key, local_file_path)
        print(f"Downloaded {key}")
else:
    print("No objects found with the specified prefix.")

! head $download_path/$prefix